In [2]:
import os
import numpy as np
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.callbacks import ModelCheckpoint
from glob import glob
import pandas as pd
from sklearn.utils import shuffle
from tqdm import tqdm
from sklearn.externals import joblib 
from keras.models import load_model
import pickle
from functools import partial
from itertools import product


Using TensorFlow backend.


In [3]:
def w_categorical_crossentropy(y_true, y_pred, weights):
    '''medida de loss com CatEnt, a partir de uma matriz de pesos, "gabarito" e previsoes '''
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = K.reshape(y_pred_max, (K.shape(y_pred)[0], 1))
    y_pred_max_mat = K.cast(K.equal(y_pred, y_pred_max), K.floatx())
    for c_p, c_t in product(range(nb_cl), range(nb_cl)):
        final_mask += (weights[c_t, c_p] * y_pred_max_mat[:, c_p] * y_true[:, c_t])
    return K.categorical_crossentropy(y_pred, y_true) * final_mask

with open('/home/julia/Hack/hackaton-cotidiano/notebooks/loss_matrix.p', "rb") as f: 
    #carregar matriz preparada para o problema dos vinhos
    w_array = pickle.load(f)

w_array = w_array.astype(np.float32)

weighted_cat_crossentropy = partial(w_categorical_crossentropy, weights=w_array)
weighted_cat_crossentropy.__name__ ='weighted_cat_crossentropy'

In [ ]:
#loss = 'categorical_crossentropy'
loss = weighted_cat_crossentropy
print("Begin")

def create_symlinks():
    ''' organizacao de pastas para o formato do keras '''
    path = "/media/5TB/Hackathon/data/food-101/images"
    df = pd.DataFrame(glob(path+"/**/*.jpg"), columns = ["filename"])
    df["label"] = df.filename.apply(lambda x: os.path.basename(os.path.dirname(x)))
    print(np.unique(df.label, return_counts=True))
    df = shuffle(df)
    df = df.reset_index(drop=True)
    train_ids = []
    val_ids = []
    test_ids = []
    train_split= int(0.75*1000)
    val_split = int(0.15*1000)

    for label in np.unique(df.label):
        current_df = df[df.label == label]
        train_ids.extend(current_df.index[0:train_split])
        val_ids.extend(current_df.index[train_split:train_split+val_split])
        test_ids.extend(current_df.index[train_split+val_split:])
        #for index, row in df[df.label == label].iterrows():
        #    print(df[df.label == label].shape)
    
    print(len(train_ids), len(val_ids), len(test_ids))
    assert len(train_ids) + len(val_ids) + len(test_ids) == len(df)
    
    basepath = "/media/5TB/Hackathon/data/food-101-keras"
    os.makedirs(basepath, exist_ok=True)

    for ids_list, split_name in zip([train_ids, val_ids, test_ids], ["train", "val", "test"]):
        print(split_name, len(ids_list))
        current_df = df[df.index.isin(ids_list)]

        for index, row in tqdm(current_df.iterrows(), total=current_df.shape[0]):
            dst_link = os.path.join(basepath, split_name, os.path.basename(os.path.dirname(row.filename)), os.path.basename(row.filename))
            os.makedirs(os.path.dirname(dst_link), exist_ok=True)
            #print(dst_link)
            os.symlink(row.filename, dst_link)

#np.unique(df[df.index.isin(train_ids)].label, return_counts=True)

np.random.seed(20) #seed para reproducao


#optimizer
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#modelo pre treinado para transfer learning
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None)

#arquitetura do final
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(101, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

for layer in base_model.layers[:-8]:
    layer.trainable = False

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model_filename = "weights/resnet_food_wloss_v0.3.h5"

#model_filename = "weights/resnet_food_v0.2.h5"
model.load_weights(model_filename)
model_filename = "weights/resnet_food_wloss_v0.4.h5"

#model_filename = "~/weights/resnet_food.h5"
#model.summary()
print("Done loading")


# In[15]:


target_size = (224, 224)

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  width_shift_range=0.2,
    # augmentation para criar invariancia com escala, rotacao, ruidos diversos 
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.2,
    horizontal_flip=True)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_path = "/home/julia/Hack/resized_data_splits/train"
val_path= "/home/julia/Hack/resized_data_splits/val"
test_path = "/home/julia/Hack/resized_data_splits/test"


#flow de treino/teste
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=64,
        class_mode='categorical', follow_links=True)

validation_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=target_size,
        batch_size=64,
        class_mode='categorical', follow_links=True)

model_checkpoint = ModelCheckpoint(model_filename, monitor='val_loss', save_best_only=True, verbose=1)
history = model.fit_generator( #checkpoint salvando apenas para melhoras
        train_generator,
        steps_per_epoch=150,
        epochs=1000,
        validation_data=validation_generator,
        validation_steps=10, callbacks=[model_checkpoint])

model_final_filename = model_filename + "_final"
model.save(model_final_filename)

print("Done")


# In[ ]:


#class_dict = {v: k for k, v in train_generator.class_indices.items()}
#joblib.dump(class_dict, "class_dict.pkl")
'''
def get_label(output):
    output = np.argmax(output)
    return class_dict[output]
'''

# In[34]:


'''
img_path = '/media/julia/5TB/Hackathon/data/food-101-keras/train/apple_pie/1818676.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
out = model.predict(x)
label = get_label(out)
print(label)
'''

